In [1]:
# Read from Cosmos DB analytical store into a Spark DataFrame and display 10 rows from the DataFrame
# To select a preferred list of regions in a multi-region Cosmos DB account, add .option("spark.cosmos.preferredRegions", "<Region1>,<Region2>")

df = spark.read\
    .format("cosmos.olap")\
    .option("spark.synapse.linkedService", "prdb")\
    .option("spark.cosmos.container", "prcol")\
    .load()

display(df.limit(10))

StatementMeta(testspark, 22, 2, Finished, Available)

SynapseWidget(Synapse.DataFrame, 90d8a14a-191b-4f4c-b1af-4e44ecc53078)

In [4]:
from pyspark.sql import DataFrame, SQLContext
sqlCtx = SQLContext(sc)

configuration = {
  "spark.synapse.linkedService": "prdb",
  "spark.cosmos.container": "prcol",
  "spark.cosmos.timetravel.timestampAsOf": "2023-12-14 05:00:00"
  }
                                                                                           
cosmosOlapTimeTravel = sc._jvm.com.microsoft.azure.cosmos.analytics.spark.connector.datasource.CosmosOlapTimeTravel
old_df = DataFrame(cosmosOlapTimeTravel.load(configuration), sqlCtx)
display(old_df)

StatementMeta(testspark, 22, 5, Finished, Available)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
/opt/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.


SynapseWidget(Synapse.DataFrame, 9c66372b-9587-4133-9d74-0619890a5967)

In [5]:
current_df = spark.read\
    .format("cosmos.olap")\
    .option("spark.synapse.linkedService", "prdb")\
    .option("spark.cosmos.container", "prcol")\
    .load()

display(current_df)

StatementMeta(testspark, 22, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 1df60a31-285c-4a0d-86a2-7a5a32722e8e)

In [7]:
changed_records = old_df.exceptAll(current_df)
display(changed_records)

StatementMeta(testspark, 22, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3ba73443-d61e-4dc3-aabf-578b161ba5f8)

In [11]:
from pyspark.sql import functions as F


# Perform an inner join on the unique identifier
joined_df = old_df.alias('a').join(current_df.alias('b'), F.col('a.id') == F.col('b.id'))

changed_records = None

# Loop through each column
for column in ["name", "score"]:
    # Filter for rows where the column value in df1 is not equal to the column value in df2
    changed_in_column = joined_df.filter(F.col('a.' + column) != F.col('b.' + column))
    
    # Union all the results
    if changed_records is None:
        changed_records = changed_in_column
    else:
        changed_records = changed_records.union(changed_in_column)

# Show the changed records
display(changed_records)

StatementMeta(testspark, 22, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, 2300c530-9c5f-4604-b307-03481d6c3575)

In [ ]:
# Find the records present in df1 but not in df2
missing_records = df1.subtract(df2)
missing_records.show()
